# Homework 2

## 2.1

In [36]:
############################
# spamFilter.py implements a spam filter based on Paul
# Graham's statistical algorithm.
# 
# Exercise 1 for Homework 2 for CS344 at Calvin University
# @author isa3
# @version March 9, 2020
############################

'''
calculate_word_probabilities returns a dictionary of strings
mapped to their float likelihood of being in a spam email

@params spamCorpus, nonSpamCorpus
    lists of lists of strings, representing example spam
    and non-spam messages respectively
@return a dictionary of strings mapped to floats
'''
def calculate_word_probabilities(spamCorpus, nonSpamCorpus):
    # spam hash table, tokens to # of occurrences
    spamDict = {}
    # nonspam hash table, tokens to # of occurrences
    nonSpamDict = {}
    # hash table mapping token to probability of spam
    probabilityDict = {}

    # tally occurrences of tokens in spam corpus
    for message in spamCorpus:
        for token in message:
            token = token.lower()
            if token in spamDict.keys():
                spamDict[token] += 1
            else:
                spamDict[token] = 1;
            # add each token as a key to the probability dict
            if token not in probabilityDict.keys():
                probabilityDict[token] = 0

    # tally occurrences of tokens in nonspam corpus
    for message in nonSpamCorpus:
        for token in message:
            token = token.lower()
            if token in nonSpamDict.keys():
                nonSpamDict[token] += 1
            else:
                nonSpamDict[token] = 1;
            # add any nonspam tokens that haven't been added already
            if token not in probabilityDict.keys():
                probabilityDict[token] = 0

    # calculate probabilities
    for key in probabilityDict.keys():
        if key in nonSpamDict.keys():
            g = 2 * nonSpamDict[key]
        else:
            g = 0
        if key in spamDict.keys():
            b = spamDict[key]
        else:
            b = 0

        if g + b > 1:
            probabilityDict[key] = max(0.01, min(0.99, min(1.0, b/len(spamCorpus)) / (min(1.0, g/len(nonSpamCorpus)) + min(1.0, b/len(spamCorpus)))))
        else:
            probabilityDict[key] = 0

    return probabilityDict

'''
calculate_message_spam_probability returns a float
representing the probability of the message parameter being
spam based on the probabilities parameter

@params message A list of strings representing a message
@params probabilities A dictionary of strings mapped to their
    respective probabilities of being in a spam message

@return a float representing the probability of message being spam
'''
def calculate_message_spam_probability(message, probabilities):
    # initial product of probabilities
    prod = 1.0
    # initial product of inverse probabilities
    inverse_prod = 1.0
    # calculate final product and inverse product values
    for token in message:
        token = token.lower()
        if token in probabilities.keys():
            prod *= probabilities[token]
            inverse_prod *= 1.0 - probabilities[token]
            
    return prod / (prod + inverse_prod)

This is a Bayesian approach to spam because it begins with a prior probability for a message to be spam (1.0) and then modifies that probability based on the likelihood calculated through data. This results in a posterior probability of the message being spam.

In [37]:
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]
spam_filter_probabilities = calculate_word_probabilities(spam_corpus, ham_corpus)

for key in spam_filter_probabilities.keys():
    print(key, ": ", spam_filter_probabilities[key])

test_message = ["i", "do", "hate", "spam", "i", "am"]

print("Probability of 'i do hate spam i am' being spam:")
print(calculate_message_spam_probability(test_message, spam_filter_probabilities))

test_message2 = ["spam", "ham", "confused", "i", "do", "like", "i"]

print("Probability of 'spam ham confused i do like i' being spam:")
print(calculate_message_spam_probability(test_message2, spam_filter_probabilities))

i :  0.5
am :  0.99
spam :  0.99
do :  0.3333333333333333
not :  0
like :  0.3333333333333333
that :  0
spamiam :  0
green :  0.01
eggs :  0.01
and :  0.01
ham :  0.01
Probability of 'i do hate spam i am' being spam:
0.9997959808221973
Probability of 'spam ham confused i do like i' being spam:
0.1999999999999998


## 2.2

a.

In [38]:
'''
This module implements the Bayesian network for Figure 14.12
in the textbook.

Written for part 2 of homework 2 for CS344 at Calvin University.

@author: isa3
@version March 8, 2020
'''

import sys
sys.path.append('/home/isa3/cs344-code/tools/aima')

from probability import BayesNet, enumeration_ask

# Utility variables
T, F = True, False

wet = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.10, F: 0.50}),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.20}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0.0})
    ])

b.

There are no independent values in the full joint probability distribution for this domain.

In [39]:
print("Showing that WetGrass is not absolutely independent of the other variables.")
print("P(WetGrass):\t\t\t\t", enumeration_ask('WetGrass', dict(), wet).show_approx())
print("P(WetGrass | Cloudy, Rain, Sprinkler):\t", enumeration_ask('WetGrass', dict(Cloudy=T, Rain=T, Sprinkler=T), wet).show_approx())

print("Showing that Rain is not absolutely independent of the other variables.")
print("P(Rain):\t\t\t\t", enumeration_ask('Rain', dict(), wet).show_approx())
print("P(Rain | Cloudy, Sprinkler, WetGrass):\t", enumeration_ask('Rain', dict(Cloudy=T, Sprinkler=T, WetGrass=T), wet).show_approx())

print("Showing that Sprinkler is not absolutely independent of the other variables.")
print("P(Sprinkler):\t\t\t\t",enumeration_ask('Sprinkler', dict(), wet).show_approx())
print("P(Sprinkler | Cloudy, Rain, WetGrass):\t", enumeration_ask('Sprinkler', dict(Cloudy=T, Rain=T, WetGrass=T), wet).show_approx())

print("Showing that Cloudy is not absolutely independent of the other variables.")
print("P(Cloudy):\t\t\t\t", enumeration_ask('Cloudy', dict(), wet).show_approx())
print("P(Cloudy | WetGrass, Rain, Sprinkler):\t", enumeration_ask('Cloudy', dict(WetGrass=T, Rain=T, Sprinkler=T), wet).show_approx())

Showing that WetGrass is not absolutely independent of the other variables.
P(WetGrass):				 False: 0.353, True: 0.647
P(WetGrass | Cloudy, Rain, Sprinkler):	 False: 0.01, True: 0.99
Showing that Rain is not absolutely independent of the other variables.
P(Rain):				 False: 0.5, True: 0.5
P(Rain | Cloudy, Sprinkler, WetGrass):	 False: 0.185, True: 0.815
Showing that Sprinkler is not absolutely independent of the other variables.
P(Sprinkler):				 False: 0.7, True: 0.3
P(Sprinkler | Cloudy, Rain, WetGrass):	 False: 0.891, True: 0.109
Showing that Cloudy is not absolutely independent of the other variables.
P(Cloudy):				 False: 0.5, True: 0.5
P(Cloudy | WetGrass, Rain, Sprinkler):	 False: 0.556, True: 0.444


c.

In [40]:
print("Showing that given Rain and Sprinkler, WetGrass is independent of Cloudy.")
print(enumeration_ask('WetGrass', dict(Rain=T, Sprinkler=T), wet).show_approx())
print(enumeration_ask('WetGrass', dict(Cloudy=T, Rain=T, Sprinkler=T), wet).show_approx())

#    Given Cloudy, Rain is independent of Sprinkler.

print("Showing that given Cloudy, Rain is independent of Sprinkler.")
print(enumeration_ask('Rain', dict(Cloudy=T), wet).show_approx())
print(enumeration_ask('Rain', dict(Cloudy=T, Sprinkler=T), wet).show_approx())

#    Given Cloudy, Sprinkler is independent of Rain.

print("Showing that given Cloudy, Sprinkler is independent of Rain.")
print(enumeration_ask('Sprinkler', dict(Cloudy=T), wet).show_approx())
print(enumeration_ask('Sprinkler', dict(Cloudy=T, Rain=T), wet).show_approx())

# Given Rain and Sprinkler, Cloudy is independent of WetGrass.
print("Showing that given Rain and Sprinkler, Cloudy is independent of WetGrass.")
print(enumeration_ask('Cloudy', dict(Rain=T, Sprinkler=T), wet).show_approx())
print(enumeration_ask('Cloudy', dict(WetGrass=T, Rain=T, Sprinkler=T), wet).show_approx())

Showing that given Rain and Sprinkler, WetGrass is independent of Cloudy.
False: 0.01, True: 0.99
False: 0.01, True: 0.99
Showing that given Cloudy, Rain is independent of Sprinkler.
False: 0.2, True: 0.8
False: 0.2, True: 0.8
Showing that given Cloudy, Sprinkler is independent of Rain.
False: 0.9, True: 0.1
False: 0.9, True: 0.1
Showing that given Rain and Sprinkler, Cloudy is independent of WetGrass.
False: 0.556, True: 0.444
False: 0.556, True: 0.444


d.

In [41]:
print("i. P(Cloudy)")
print(enumeration_ask('Cloudy', dict(), wet).show_approx())

i. P(Cloudy)
False: 0.5, True: 0.5


P(Cloudy) = < 0.5, 0.5 >

In [42]:
print("ii. P(Sprinkler | Cloudy)")
print(enumeration_ask('Sprinkler', dict(Cloudy=T), wet).show_approx())

ii. P(Sprinkler | Cloudy)
False: 0.9, True: 0.1


P(Sprinkler | Cloudy) = < 0.10, 0.90 >

In [43]:
print("iii. P(Cloudy | Sprinkler ^ !Rain)")
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), wet).show_approx())

iii. P(Cloudy | Sprinkler ^ !Rain)
False: 0.952, True: 0.0476


P(Cloudy | Sprinkler ^ !Rain) = alpha P(Cloudy, Sprinkler, !Rain)

                          = alpha P(Cloudy) * P(Sprinkler | Cloudy) * P(!Rain | Cloudy)
                          
                          = alpha < P(Cloudy) * P(Sprinkler | Cloudy) * P(!Rain | Cloudy)
                          
                                  P(!Cloudy) * P(Sprinkler | !Cloudy) * P(!Rain | !Cloudy) >
                                  
                          = alpha < 0.5 * 0.1 * 0.2, 0.5 * 0.5 * 0.8 >
                          
                          = alpha < .01, .20 >
                          
                          = < .048, .952 >

In [44]:
print("iv. P(WetGrass | Cloudy ^ Sprinkler ^ Raining)")
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), wet).show_approx())

iv. P(WetGrass | Cloudy ^ Sprinkler ^ Raining)
False: 0.01, True: 0.99


P(WetGrass | Cloudy ^ Sprinkler ^ Raining) = P(WetGrass | Sprinkler ^ Raining) 

                                            = < 0.99, 0.01 >

In [45]:
print("v. P(Cloudy | !WetGrass)")
print(enumeration_ask('Cloudy', dict(WetGrass=F), wet).show_approx())

v. P(Cloudy | !WetGrass)
False: 0.639, True: 0.361


P(Cloudy | !Wetgrass) = 

    P(Cloudy) * 

    ( [ P(Rain | Cloudy) * P(!Wet  | Rain) * P(Sprinkler | Cloudy) * P(!Wet | Sprinkler) ] +

      [ P(!Rain | Cloudy) * P(!Wet  | !Rain) * P(Sprinkler | Cloudy) * P(!Wet | Sprinkler) ] +
  
      [ P(Rain | Cloudy) * P(!Wet  | Rain) * P(!Sprinkler | Cloudy) * P(!Wet | !Sprinkler) ] +
  
      [ P(!Rain | Cloudy) * P(!Wet  | !Rain) * P(!Sprinkler | Cloudy) * P(!Wet | !Sprinkler) ]
                                                                                  )
      
      
= alpha < P(Cloudy) * 

    ( [ P(Rain | Cloudy) * P(Sprinkler | Cloudy) * P(!Wet | Rain ^ Sprinkler) ] +

      [ P(!Rain | Cloudy) * P(Sprinkler | Cloudy) * P(!Wet | !Rain ^ Sprinkler) ] +
  
      [ P(Rain | Cloudy) * P(!Sprinkler | Cloudy) * P(!Wet | Rain ^ !Sprinkler) ] +
  
      [ P(!Rain | Cloudy) * P(!Sprinkler | Cloudy) * P(!Wet | !Rain ^ !Sprinkler) ] ) ,
  
    P(!Cloudy) * 

    ( [ P(Rain | !Cloudy) * P(Sprinkler | !Cloudy) * P(!Wet | Rain ^ Sprinkler) ] +

      [ P(!Rain | !Cloudy) * P(Sprinkler | !Cloudy) * P(!Wet | !Rain ^ Sprinkler) ] +
  
      [ P(Rain | !Cloudy) * P(!Sprinkler | !Cloudy) * P(!Wet | Rain ^ !Sprinkler) ] +
  
      [ P(!Rain | !Cloudy) * P(!Sprinkler | !Cloudy) * P(!Wet | !Rain ^ !Sprinkler) ] ) >
  


= alpha < 0.5 *

    ( [ .8 * .1 * .01 ] +

      [ .2 * .1 * .1 ] +
  
      [ .8 * .9 * .1 ] +
  
      [ .2 * .9 * 1.0 ] ) ,
  
      0.5 *
  
    ( [ .2 * .5 * .01 ] +

      [ .8 * .5 * .1 ] +
  
      [ .2 * .5 * .1] +
  
      [ .8 * .5 * 1.0] ) >
  

= alpha < .1274, 0.2255 >

    alpha = 1 / 0.3529
    
= <  0.361 , 0.639 >